In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from nltk.probability import FreqDist
import nltk
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [38]:
# 测试样本数
N = 90


def load_user_cmd_new(filename):
    cmd_list = []
    dist = []
    with open(filename) as f:
        i = 0
        x = []
        for line in f:
            line = line.strip('\n')
            x.append(line)
            dist.append(line)
            i += 1
            if i == 100:
                cmd_list.append(x)
                x = []
                i = 0

    fdist = FreqDist(dist)  # 按照频率排序
    # for i in fdist:
    #     print(i, fdist[i])
    return cmd_list, fdist


def load_user_cmd(filename):
    cmd_list = []
    dist_max = []
    dist_min = []
    dist = []
    with open(filename) as f:
        i = 0
        x = []
        for line in f:
            line = line.strip('\n')
            x.append(line)
            dist.append(line)
            i += 1
            if i == 100:
                cmd_list.append(x)
                x = []
                i = 0

    fdist = FreqDist(dist).keys()
    dist_max = set(fdist[0:50])
    dist_min = set(fdist[-50:])
    return cmd_list, dist_max, dist_min


def get_user_cmd_feature(user_cmd_list, dist_max, dist_min):
    user_cmd_feature = []
    for cmd_block in user_cmd_list:
        f1 = len(set(cmd_block))
        fdist = FreqDist(cmd_block).keys()
        f2 = fdist[0:10]
        f3 = fdist[-10:]
        f2 = len(set(f2) & set(dist_max))
        f3 = len(set(f3) & set(dist_min))
        x = [f1, f2, f3]
        user_cmd_feature.append(x)
    return user_cmd_feature


def get_user_cmd_feature_new(user_cmd_list, dist):
    '''
    user_cmd_list:150组,每组100个
    dist:按照频率降序,107个
    '''
    user_cmd_feature = []

    for cmd_list in user_cmd_list:
        # 将每组的100个循环
        v = [0]*len(dist)
        for i in range(0, len(dist)):
            if dist[i] in cmd_list:
                #如果最常见的内容在该组出现，将该值加一
                v[i] += 1
        user_cmd_feature.append(v)

    return user_cmd_feature


def get_label(filename, index=0):
    x = []
    with open(filename) as f:
        for line in f:
            line = line.strip('\n')
            # 按照空格切片，取每个的第index个，
            # label.txt中，每行50个(对应50个user)，一共100行
            x.append(int(line.split()[index]))
    return x



In [7]:
user_cmd_list, dist = load_user_cmd_new("../../Data/MasqueradeDat/User3")
# user_cmd_list:一共150组,每组100个
# dist: 107个,按照频率降序排列
print(len(dist))

107


In [78]:
# user_cmd_feature共150组，每组107个0
user_cmd_feature = get_user_cmd_feature_new(user_cmd_list, dist)
# labels为label文件中的第三列所有
labels = get_label("../../Data/MasqueradeDat/label.txt", 2)
y = [0]*50+labels#在labels前加了50个0

x_train = user_cmd_feature[0:N]
y_train = y[0:N]

x_test = user_cmd_feature[N:150]
y_test = y[N:150]

neigh = KNeighborsClassifier(n_neighbors=3)
# 使用x_train为训练数据，y_train为目标值
neigh.fit(x_train, y_train)
# predict函数为预测函数
y_predict = neigh.predict(x_test)

score = np.mean(y_test == y_predict)*100

print('y_test\n',y_test)
print('y_predict\n',y_predict)
print('score\n',score)
# 精确度(precision)=正确预测的个数(TP)/预测为正样本的个数(TP+FP)
# 召回率(recall)=正确预测值的个数(TP)/实际为正样本的个数(TP+FN)
# F1值=2*精度*召回率/(精度+召回率)
# support列为每个标签的出现次数(权重)
# macro avg：每个类别指标中的未加权平均值(一列)
# eighted avg：每个类别指标中的加权平均
print('classification_report(y_test, y_predict)\n',classification_report(y_test, y_predict))
print('metrics.confusion_matrix(y_test, y_predict)\n',metrics.confusion_matrix(y_test, y_predict))

y_test
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y_predict
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
score
 83.33333333333334
classification_report(y_test, y_predict)
               precision    recall  f1-score   support

           0       0.83      1.00      0.91        50
           1       0.00      0.00      0.00        10

    accuracy                           0.83        60
   macro avg       0.42      0.50      0.45        60
weighted avg       0.69      0.83      0.76        60

metrics.confusion_matrix(y_test, y_predict)
 [[50  0]
 [10  0]]
